In [1]:

from torchvision.models.segmentation import fcn_resnet50
import torch

def create_model():
    
    # Load the pretrained model and modify the classifier
    model = fcn_resnet50(pretrained=False)
    print(model)
    model.classifier[4] = torch.nn.Conv2d(512, 2, kernel_size=(1, 1))
    model.aux_classifier = None
        
    return model

model = create_model()

d:\NIITN_WORK\Personal_Project\Youtube_Projects\telegram_bot\Blood_Cell_Segmentation_Bot\myenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\NIITN_WORK\Personal_Project\Youtube_Projects\telegram_bot\Blood_Cell_Segmentation_Bot\myenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


FCN(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

In [2]:
import numpy as np
import cv2

image = np.array([
    [[255,   0,   0], 
     [  1, 255,   0], 
     [  2,   0, 255]],
    [[255, 255,   0], 
     [  3, 255, 255], 
     [255,   0, 255]],
    [[128, 128, 128], 
     [ 64,  64,  64], 
     [192, 192, 192]]
], dtype=np.uint8)

print(image[..., 0])
print(image[..., 1])  
print(image[..., 2])  

[[255   1   2]
 [255   3 255]
 [128  64 192]]
[[  0 255   0]
 [255 255   0]
 [128  64 192]]
[[  0   0 255]
 [  0 255 255]
 [128  64 192]]


In [3]:
import numpy as np

# Initial mask, overlay, and image
mask = np.array([
    [0, 255, 255],
    [0, 0, 255],
    [255, 0, 0]
])

overlay = np.array([[[0, 0, 0], [0, 0, 0], [0, 0, 0]],
                    [[0, 0, 0], [0, 0, 0], [0, 0, 0]],
                    [[0, 0, 0], [0, 0, 0], [0, 0, 0]]], dtype=np.uint8)

image = np.array([
    [[255,   0,   0], 
     [  0, 255,   0], 
     [  0,   0, 255]],
    [[255, 255,   0], 
     [  0, 255, 255], 
     [255,   0, 255]],
    [[128, 128, 128], 
     [ 64,  64,  64], 
     [192, 192, 192]]
], dtype=np.uint8)

alpha = 0.5
class_colors = {
    1: [255, 255, 0],
}

# Convert mask to binary
print("Before Mask ==> ", mask)
mask[mask != 0] = 1
print("After Mask ==> ", mask)


print("Before Overlay ==> ", overlay)

# Apply class colors to overlay
for class_id, color in class_colors.items():
    class_mask = (mask == class_id)
    print("Class Mask ==> ", class_mask)
    for c in range(3):
        overlay[..., c][class_mask] = color[c] # color[2]

# Extract channels
R = overlay[..., 0]
G = overlay[..., 1]
B = overlay[..., 2]
print("Overlay Red Channel ==> ", R)
print("Overlay Green Channel ==> ", G)  
print("Overlay Blue Channel ==> ", B)
print("After Overlay ==> ", overlay)


Before Mask ==>  [[  0 255 255]
 [  0   0 255]
 [255   0   0]]
After Mask ==>  [[0 1 1]
 [0 0 1]
 [1 0 0]]
Before Overlay ==>  [[[0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]]]
Class Mask ==>  [[False  True  True]
 [False False  True]
 [ True False False]]
Overlay Red Channel ==>  [[  0 255 255]
 [  0   0 255]
 [255   0   0]]
Overlay Green Channel ==>  [[  0 255 255]
 [  0   0 255]
 [255   0   0]]
Overlay Blue Channel ==>  [[0 0 0]
 [0 0 0]
 [0 0 0]]
After Overlay ==>  [[[  0   0   0]
  [255 255   0]
  [255 255   0]]

 [[  0   0   0]
  [  0   0   0]
  [255 255   0]]

 [[255 255   0]
  [  0   0   0]
  [  0   0   0]]]


In [4]:
# Blend image and overlay
blended_image = image.copy()
for c in range(3):
    image_red_channel = image[..., c].copy()
    print("image_red_channel ==> \n",image_red_channel)

    image_red_channel[mask == 1] =  image_red_channel[mask == 1] * (1 - alpha)
    print("image_mul ==> \n",image_red_channel)

    overlay_red_channel = overlay[...,c].copy()
    print("overlay_red_channel ==> \n",overlay_red_channel)
    
    overlay_red_channel[mask == 1] = overlay_red_channel[mask == 1] * alpha
    print("overlay_mul ==> \n",overlay_red_channel)

    blended_image[..., c] = image_red_channel + overlay_red_channel
    
    print("blended_red_channel ==> \n",blended_image[...,c])
    break

image_red_channel ==> 
 [[255   0   0]
 [255   0 255]
 [128  64 192]]
image_mul ==> 
 [[255   0   0]
 [255   0 127]
 [ 64  64 192]]
overlay_red_channel ==> 
 [[  0 255 255]
 [  0   0 255]
 [255   0   0]]
overlay_mul ==> 
 [[  0 127 127]
 [  0   0 127]
 [127   0   0]]
blended_red_channel ==> 
 [[255 127 127]
 [255   0 254]
 [191  64 192]]


In [5]:
# Blend image and overlay
blended_image = image.copy()
for c in range(3):

    blended_image[..., c][mask == 1] = (
        image[..., c][mask == 1] * (1 - alpha) + overlay[..., c][mask == 1] * alpha
    ).astype(np.uint8)
    
    # break

print("blended_red_channel ==> \n",blended_image[...,0])
print("blended_green_channel ==> \n",blended_image[...,1])
print("blended_blue_channel ==> \n",blended_image[...,2])

blended_red_channel ==> 
 [[255 127 127]
 [255   0 255]
 [191  64 192]]
blended_green_channel ==> 
 [[  0 255 127]
 [255 255 127]
 [191  64 192]]
blended_blue_channel ==> 
 [[  0   0 127]
 [  0 255 127]
 [ 64  64 192]]
